In [1]:
import pandas as pd
import numpy as np
from pandas import CategoricalDtype

In [2]:
# Set custom colors
nfl_red = "#d50a0a"
nfl_blue = "#013369"

In [3]:
# Import the dyadic data
df = pd.read_csv("../07_data_staged/dyadic_data.csv")

In [4]:
# Convert Time_rest to time delta
df["Time_rest"] = pd.to_timedelta(df["Time_rest"])

# Convert Time_rest to number of hours
df['Time_rest_hours'] = df['Time_rest'] / np.timedelta64(1, 'h')

# Convert number of hours to days
df['Time_rest_days'] = df['Time_rest_hours'] / 24

In [5]:
#Drop Tie
df = df[df.Game_pts_diff!=0]


#Win
df["Win"] = 1*(df.Game_pts_diff > 0)

In [6]:
# Convert categorial variables to categories
for col in ['Week','Game_type','Surface','Game_outcome','Rivalry','Same_surface','Field']: df[col] = df[col].astype("category")


In [7]:
# Time rest
# Set time rest for Week 1 as the longest time a team has to rest during the regular season
time_rest_max = df.groupby(["Season","Game_type"])["Time_rest"].max().reset_index(name="Time_rest_max")
time_rest_max = time_rest_max.loc[time_rest_max.Game_type=="Regular"]
time_rest_max.drop(columns="Game_type", inplace=True)

# Merge to df
df = pd.merge(df,time_rest_max, on=["Season"], how = "left")

# Fill NA
df["Time_rest"] = df["Time_rest"].fillna(df["Time_rest_max"])

# Convert Time_rest to number of hours
df['Time_rest_hours'] = df['Time_rest'] / np.timedelta64(1, 'h')

# Convert number of hours to days
df['Time_rest_days'] = df['Time_rest_hours'] / 24

# Drop Time_rest_max
df.drop(columns = "Time_rest_max", inplace = True)

In [8]:
df.loc[df["Miles_traveled"]>0, 'Miles_traveled'].quantile()
dir(df.loc[df["Miles_traveled"]>0, 'Miles_traveled'])

# Miles
median_miles = df.loc[df["Miles_traveled"]>0, 'Miles_traveled'].median()
df["Miles"] = np.where(df['Miles_traveled']==0,0,np.where(df['Miles_traveled'] <= median_miles,1,2))
df.Miles.value_counts()

0    7201
1    3680
2    3679
Name: Miles, dtype: int64

In [9]:
# Timerest
df["Time_rest_ordinal"] = np.where(df["Time_rest_days"] == 7, 1, np.where(df["Time_rest_days"] < 7, 0, 2))


In [10]:
rival = ['No','Conference','Division']
rival_types = CategoricalDtype(categories=rival, ordered=True)
df['Rivalry'] = df['Rivalry'].astype(rival_types)
df['Rivalry'] = df['Rivalry'].cat.codes

In [11]:
df['Division_Rival'] = 1*(df["Rivalry"] == 2)
df['Conference_Rival'] = 1*(df["Rivalry"] == 1)

In [12]:
# Grass
df["Grass"] = 1*(df["Surface"] == "Grass")

# Home
df["Home"] = 1*(df["Field"] == "Home")

# Same surface as the team's home field
df["Same_surface"] = 1*(df["Same_surface"] == "Yes")


In [13]:
# Order the weeks
df["Week"].unique()
week = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','WildCard','Division','ConfChamp','SuperBowl']

week_types = CategoricalDtype(categories=week, ordered=True)
df['Week'] = df['Week'].astype(week_types)
df['Week'] = df['Week'].cat.codes


In [14]:
playoff = df.loc[df.Game_type != "Regular"]
reg = df.loc[df.Game_type == "Regular"]

In [15]:
reg = reg[['Week', 'Season','Team_A', 'Team_B',
         'Game_TO','Game_yds', 'Game_pts_diff', 'Game_yards_diff',
         'Time_rest_hours', 'Miles_traveled',
         'Division_Rival','Conference_Rival',
         'Grass', 'Home','Time_diff','Same_surface', 'Attendance', 'Capacity','Attendance_pct',
         'Season_SRS', 'Season_SoS', 'Season_offense','Season_defense',
        'Win']]

In [16]:
reg.to_csv("../07_data_staged/clean_data.csv", index=True, index_label='id')